In [60]:
import pandas as pd
import numpy as np
import ast

In [61]:
df2 = pd.read_csv('Dataset/movies_dataset.csv')


C:\Users\germa\AppData\Local\Temp\ipykernel_17472\822426263.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv('Dataset/movies_dataset.csv')


In [62]:
# Eliminamos las columnas que no sirven

df2.drop(columns=['video','imdb_id','adult','original_title','poster_path', 'homepage'], inplace=True)

In [63]:
df2.dropna(subset=['release_date'], inplace=True)

In [64]:
df2['release_date'].isna().sum()

0

In [65]:
df2['revenue'].isna().sum()

3

In [66]:
# Los valores nulos de los campos revenue, budget deben ser rellenados por el número 0.

df2['revenue'] = df2['revenue'].apply(lambda x: x if pd.notnull(x) else 0)

df2['budget'] = df2['budget'].apply(lambda x: x if pd.notnull(x) else 0)
df2['budget'] = pd.to_numeric(df2['budget'], errors='coerce').fillna(0)



In [67]:
df2['revenue'].isna().sum()

0

In [68]:
df2['budget'].value_counts()

budget
0.0            36493
5000000.0        286
10000000.0       259
20000000.0       243
2000000.0        242
               ...  
270000000.0        1
923.0              1
72500000.0         1
2160000.0          1
1254040.0          1
Name: count, Length: 1223, dtype: int64

In [69]:
df2['return'] = df2.apply(lambda x: (x['revenue'] / x['budget']) if x['budget'] != 0 else 0, axis=1)

In [70]:
df2['release_date'] = pd.to_datetime(df2['release_date'], errors='coerce')
df2['release_date'] = df2['release_date'].dt.strftime('%Y-%m-%d')
df2.dropna(subset= ['release_date'], inplace=True)

df2['release_date'] = pd.to_datetime(df2['release_date'])
df2['release_year'] = df2['release_date'].dt.year


In [71]:

df2['belongs_to_collection'] = df2['belongs_to_collection'].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else np.nan)
df2['belongs_to_collection'] = df2['belongs_to_collection'].apply(lambda x: x['name'] if isinstance(x, dict) else np.nan)

df2['production_companies'] = df2['production_companies'].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else np.nan)
df2['production_companies'] = df2['production_companies'].apply(lambda x: ', '.join([i['name'] for i in x]) if isinstance(x, list) else np.nan)

df2['genres'] = df2['genres'].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else np.nan)
df2['genres'] = df2['genres'].apply(lambda x: ', '.join([i['name'] for i in x]) if isinstance(x, list) else np.nan)

df2['spoken_languages'] = df2['spoken_languages'].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else np.nan)
df2['spoken_languages'] = df2['spoken_languages'].apply(lambda x: ', '.join([i['name'] for i in x]) if isinstance(x, list) else np.nan)

df2['production_countries'] = df2['production_countries'].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else np.nan)
df2['production_countries'] = df2['production_countries'].apply(lambda x: ', '.join([i['name'] for i in x]) if isinstance(x, list) else np.nan)

In [74]:
df1 = pd.read_csv('Dataset/credits.csv')


In [75]:
df1.drop(columns=['cast'], inplace=True)
#df1['cast'] = df1['cast'].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else np.nan)
#df1['cast'] = df1['cast'].apply(lambda x: ', '.join([i['name'] for i in x]) if isinstance(x, list) else np.nan)

In [76]:
df1['crew'] = df1['crew'].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else np.nan)
df1['crew'] = df1['crew'].apply(lambda x: ', '.join([i['name'] for i in x if i['job'] == 'Director']) if isinstance(x, list) else np.nan)


In [77]:
df2['id'] = df2['id'].astype('int64')

In [78]:
data = df2.merge(df1, how='left', on='id')


In [79]:
data['title'] = data['title'].apply(lambda x: str(x).lower().capitalize().strip())
data

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,return,release_year,crew
0,Toy Story Collection,30000000.0,"Animation, Comedy, Family",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,Pixar Animation Studios,United States of America,1995-10-30,...,81.0,English,Released,NaN,Toy story,7.7,5415.0,12.451801,1995,John Lasseter
1,NaN,65000000.0,"Adventure, Fantasy, Family",8844,en,When siblings Judy and Peter discover an encha...,17.015539,"TriStar Pictures, Teitler Film, Interscope Com...",United States of America,1995-12-15,...,104.0,"English, Français",Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,4.043035,1995,Joe Johnston
2,Grumpy Old Men Collection,0.0,"Romance, Comedy",15602,en,A family wedding reignites the ancient feud be...,11.7129,"Warner Bros., Lancaster Gate",United States of America,1995-12-22,...,101.0,English,Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier old men,6.5,92.0,0.000000,1995,Howard Deutch
3,NaN,16000000.0,"Comedy, Drama, Romance",31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,Twentieth Century Fox Film Corporation,United States of America,1995-12-22,...,127.0,English,Released,Friends are the people who let you be yourself...,Waiting to exhale,6.1,34.0,5.090760,1995,Forest Whitaker
4,Father of the Bride Collection,0.0,Comedy,11862,en,Just when George Banks has recovered from his ...,8.387519,"Sandollar Productions, Touchstone Pictures",United States of America,1995-02-10,...,106.0,English,Released,Just When His World Is Back To Normal... He's ...,Father of the bride part ii,5.7,173.0,0.000000,1995,Charles Shyer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45447,NaN,0.0,"Drama, Action, Romance",30840,en,"Yet another version of the classic epic, with ...",5.683753,"Westdeutscher Rundfunk (WDR), Working Title Fi...","Canada, Germany, United Kingdom, United States...",1991-05-13,...,104.0,English,Released,NaN,Robin hood,5.7,26.0,0.000000,1991,John Irvin
45448,NaN,0.0,Drama,111109,tl,An artist struggles to finish his work while a...,0.178241,Sine Olivia,Philippines,2011-11-17,...,360.0,,Released,NaN,Century of birthing,9.0,3.0,0.000000,2011,Lav Diaz
45449,NaN,0.0,"Action, Drama, Thriller",67758,en,"When one of her hits goes wrong, a professiona...",0.903007,American World Pictures,United States of America,2003-08-01,...,90.0,English,Released,A deadly game of wits.,Betrayal,3.8,6.0,0.000000,2003,Mark L. Lester
45450,NaN,0.0,,227506,en,"In a small town live two brothers, one a minis...",0.003503,Yermoliev,Russia,1917-10-21,...,87.0,,Released,NaN,Satan triumphant,0.0,0.0,0.000000,1917,Yakov Protazanov


In [80]:
#data.to_csv('Dataset/clean_data.csv', index=False)